## Testing our Conversational Language Understanding Model with API Calls

### Installing Libraries and Utilities

In [ ]:
%pip install azure-ai-language-conversations==1.1.0 azure-identity

### Setting up the Environment

In [ ]:
import os
import requests
import json
from dotenv import load_dotenv

load_dotenv()
language_endpoint = os.getenv("LANGUAGE_ENDPOINT")
language_key = os.getenv("LANGUAGE_KEY")
project_name = os.getenv("PROJECT_NAME")
deployment_name = os.getenv("DEPLOYMENT_NAME")
weather_api_key = os.getenv("WEATHER_API_KEY")

### Creating the CLU Client

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.language.conversations import ConversationAnalysisClient

client = ConversationAnalysisClient(
    endpoint=language_endpoint,
    credential=AzureKeyCredential(language_key)
)

### Defining the Weather Function

In [ ]:
def get_weather(location):
    #making a call to the geocoding API to fetch the coordinates of a particular location
    geocoding_url = 'http://api.openweathermap.org/geo/1.0/direct?q='+ str(location) +'&limit=5&appid=' +str(weather_api_key)
    geocoding_response=requests.get(geocoding_url).json()
    latitude = geocoding_response[0]['lat']
    longitude=geocoding_response[0]['lon']

    #making a call to the weather API 
    weather_url='https://api.openweathermap.org/data/2.5/weather?lat=' +str(latitude) +'&lon=' +str(longitude) +'&appid=' +str(weather_api_key)
    weather_response=requests.get(weather_url).json()
    weather=weather_response['weather'][0]['description']
    return 'weather description of {} is {}'.format(location,weather)

### Starting the Chat Loop

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Exiting the chat. Goodbye!")
        break

    #defining the body of our RESTful query
    data = {
        "kind": "Conversation",
        "analysisInput": {
            "conversationItem": {
                "id": "1",
                "text": user_input,
                "modality": "text",
                "language": "en",
                "participantId": "1"
            }
        },
        "parameters": {
            "projectName": project_name,
            "verbose": True,
            "deploymentName": deployment_name,
            "stringIndexType": "TextElement_V8"
        }
    }

    #making a call to our NLU model published with a prediction URL.
    result = client.analyze_conversation(task=data)

    top_intent =result['result']['prediction']['topIntent']

    location=result['result']['prediction']['entities'][0]['text']

    if top_intent.lower() == 'getweather':
        weather_info = get_weather(location)
        print("Bot:", weather_info)

    else:
        print("Bot: I'm sorry, I can only provide weather information at the moment.")


